In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# EDA

#### For the preparations, let's first import the necessary libraries and load the necessary files for our EDA.
#### Hazırlıklar için öncelikle gerekli kütüphaneleri import edelim ve EDA'mız için gerekli dosyaları yükleyelim.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df.head()

In [ ]:
df.info()

### From this information we can already see that some features (like Alley and PoolQC) will not be relevant to our exploratory analysis as there are too many missing values. Also, there are so many features to analyze that it may be better to focus on the ones that can give us real information. Let's remove the Id and features with a NaN value of 30% or less.

### Bu bilgilerden, çok fazla eksik değer olduğundan (Alley ve PoolQC gibi) bazı özelliklerin keşif analizimizle ilgili olmayacağını şimdiden görebiliriz. Ayrıca, analiz edilecek o kadar çok özellik var ki, bize gerçek bilgiler verebilecek olanlara odaklanmak daha iyi olabilir. Id'yi ve %30 veya daha az NaN değerine sahip özellikleri kaldıralım.

In [ ]:
# df.count(), NaN değerlerini içermez
df2 = df[[column for column in df if df[column].count() / len(df) >= 0.3]]
del df2['Id']
print("Bırakılan sütunların listesi:", end=" ")
for c in df.columns:
    if c not in df2.columns:
        print(c, end=", ")
print('\n')

In [ ]:
df = df2

#### Note: If we take the features we just removed and look at their descriptions in the "data_description.txt" file, we can deduce that these features may not be available in all houses (which explains the "NaN" values). We can convert them to categorical dummy values ​​in our next Data preparation/cleaning notebook.

#### Not: Az önce kaldırdığımız özellikleri alır ve "data_description.txt" dosyasındaki açıklamalarına bakarsak, bu özelliklerin tüm evlerde bulunmayabileceğini çıkarabiliriz (bu, "NaN" değerlerini açıklar). Bir sonraki Veri hazırlama/temizleme not defterimizde bunları kategorik kukla değerlere dönüştürebiliriz.

### Let's take a look at how house prices are distributed.
### Şimdi konut fiyatlarının nasıl dağıldığına bir göz atalım.

In [ ]:
print(df['SalePrice'].describe())
plt.figure(figsize=(20, 10))
sns.distplot(df['SalePrice'], color='g', bins=100, hist_kws={'alpha': 0.4});

### With this information we can see that prices are skewed to the right and some extreme values ​​are above ~500,000. Finally, we'll want to get rid of these to get the normal distribution of the argument ("SalePrice") for machine learning.

### Bu bilgilerle fiyatların sağa çarpık olduğunu ve bazı uç değerlerin ~500.000'in üzerinde olduğunu görebiliriz. Sonunda, makine öğrenimi için bağımsız değişkenin ("SalePrice") normal dağılımını elde etmek için bunlardan kurtulmak isteyeceğiz.

# Digital data distribution
# Sayısal veri dağıtım

### For this part, let's graph all the features and look at their distribution. To do this, let's first list all the data types in our dataset and only get the numeric ones:

### Bu kısım için, tüm özelliklerin grafiğini çizerek dağılımına bakalım. Bunu yapmak için önce veri kümemizdeki tüm veri türlerini listeleyelim ve yalnızca sayısal olanları alalım:

In [ ]:
list(set(df.dtypes.tolist()))

In [ ]:
df_num = df.select_dtypes(include = ['float64', 'int64'])
df_num.head()

In [ ]:
df_num.hist(figsize=(20, 20), bins=50, xlabelsize=8, ylabelsize=8); # ; --> matplotlib ayrıntılı bilgilerine sahip olmaktan kaçının

### Features like `1stFlrSF`, `TotalBsmtSF`, `LotFrontage`, `GrLiveArea`... seem to share a distribution similar to what we have with `SalePrice`.

### `1stFlrSF`, `TotalBsmtSF`, `LotFrontage`, `GrLiveArea`... gibi özellikler, `SalePrice` ile sahip olduğumuza benzer bir dağıtım paylaşıyor gibi görünüyor.

# Correlation
# Korelasyon 

#### Now we will try to find out which features are strongly associated with SalePrice. We will store them in a variable called golden_features_list. To do this we will reuse our df_num dataset.
#### Şimdi hangi özelliklerin SalePrice ile güçlü bir şekilde ilişkili olduğunu bulmaya çalışacağız. Onları golden_features_list adlı bir değişkende saklayacağız. Bunu yapmak için df_num veri kümemizi yeniden kullanacağız.

In [ ]:
df_num_corr = df_num.corr()['SalePrice'][:-1] # -1 çünkü en son satır SalePrice

In [ ]:
golden_features_list = df_num_corr[abs(df_num_corr) > 0.5].sort_values(ascending=False)
print("SalePrice ile güçlü bir şekilde ilişkili {} değer var:\n{}".format(len(golden_features_list), golden_features_list))


#### Excellent, we now have a list of strongly correlated values, but this list is incomplete as we know correlation is affected by outliers.
#### So we can proceed as follows:
#### Plot the numeric features and see which ones have few or explainable outliers Remove outliers from those features and see which ones can have a good correlation without outliers
#### Btw, correlation alone doesn't always explain the relationship between data so plotting them can even lead us to new insights and likewise check if our correlated values ​​have a linear relationship to SalePrice.
#### For example, relationships such as curvilinear relationships cannot be predicted just by looking at the correlation value, so let's take the features we extracted from our correlation table and plot them to see if they show some kind of pattern.


###### ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### Mükemmel, artık güçlü bir şekilde ilişkili değerlerin bir listesine sahibiz, ancak korelasyonun aykırı değerlerden etkilendiğini bildiğimiz için bu liste eksik.
#### Böylece aşağıdaki gibi ilerleyebiliriz: 
####   Sayısal özellikleri çizin ve hangilerinin çok az veya açıklanabilir aykırı değerlere sahip olduğunu görün Bu özelliklerden aykırı değerleri kaldırın ve bunların aykırı değerleri olmadan hangisinin iyi bir korelasyona sahip olabileceğini görün 
####   Btw, korelasyon tek başına veriler arasındaki ilişkiyi her zaman açıklamaz, bu nedenle onları çizmek bizi yeni içgörülere bile götürebilir ve aynı şekilde, ilişkili değerlerimizin SalePrice ile doğrusal bir ilişkisi olup olmadığını kontrol edin. 
#### Örneğin, eğrisel ilişki gibi ilişkiler, yalnızca korelasyon değerine bakarak tahmin edilemez, bu nedenle korelasyon tablomuzdan çıkardığımız özellikleri alıp bir tür model gösterip göstermediklerini görmek için çizelim.

In [ ]:
for i in range(0, len(df_num.columns), 5):
    sns.pairplot(data=df_num,
                x_vars=df_num.columns[i:i+5],
                y_vars=['SalePrice'])

#### We can clearly define some relationships. Many of them seem to have a linear relationship with the Sale Price, and if we look closely at the data, we can see that many data points are located above x = 0, which may indicate the absence of such a feature at home.
#### Let's take OpenPorchSF, I doubt all houses have patios (eg mine doesn't but someday... yes someday...) I'm not losing hope.

###### ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


#### Bazı ilişkileri açıkça tanımlayabiliriz. Birçoğunun Satış Fiyatı ile doğrusal bir ilişkisi var gibi görünüyor ve verilere yakından bakarsak, evde böyle bir özelliğin bulunmadığına işaret edebilecek birçok veri noktasının x = 0 üzerinde bulunduğunu görebiliriz. 
#### OpenPorchSF'yi ele alalım, tüm evlerin verandası olduğundan şüpheliyim (örneğin benimki yok ama bir gün... evet bir gün...) umudunu kaybetmiyorum.

##### Now let's remove these 0 values ​​and repeat the process of finding the associated values:

##### Şimdi bu 0 değerleri kaldıralım ve ilişkili değerleri bulma işlemini tekrarlayalım:

In [ ]:
import operator

In [ ]:
individual_features_df = []
for i in range(0, len(df_num.columns) - 1): # -1 çünkü son sütun SalePrice
    tmpDf = df_num[[df_num.columns[i], 'SalePrice']]
    tmpDf = tmpDf[tmpDf[df_num.columns[i]] != 0]
    individual_features_df.append(tmpDf)

In [ ]:
all_correlations = {feature.columns[0]: feature.corr()['SalePrice'][0] for feature in individual_features_df}
all_correlations = sorted(all_correlations.items(), key=operator.itemgetter(1))
for (key, value) in all_correlations:
    print("{:>15}: {:>15}".format(key, value))

#### By cleaning the data a bit, we found another strongly correlated value. Now our golden_features_list variable looks like this:

####  Verileri biraz temizleyerek güçlü bir şekilde ilişkili başka bir değer bulduk. Şimdi golden_features_list değişkenimiz şöyle görünüyor:

In [ ]:
golden_features_list = [key for key, value in all_correlations if abs(value) >= 0.5]
print("SalePrice ile güçlü bir şekilde ilişkili {} değer var:\n{}".format(len(golden_features_list), golden_features_list))

#### We found estimators strongly correlated with "SalePrice". Then with feature engineering we can add dummy values ​​where the value of a particular feature would be > 0 (existence of such a feature) and 0 would be 0. For example, for "2ndFlrSF" we can create a dummy value for its presence or absence and finally aggregate it as '1stFlrSF'.


#### "SalePrice" ile güçlü bir şekilde ilişkili tahmin ediciler bulduk. Daha sonra özellik mühendisliği ile, belirli bir özelliğin değerinin > 0 olduğu yerde 1 (böyle bir özelliğin varlığı) ve 0'ın 0 olacağı kukla değerler ekleyebiliriz. Örneğin "2ndFlrSF" için, varlığı veya yokluğu için kukla bir değer oluşturabilir ve son olarak bunu "1stFlrSF" olarak toplayabiliriz.

### Solution
#### By looking at the correlation between numeric values, we discovered 11 properties that have a strong correlation with the price of a house. Besides correlation, we did not find any notable patterns in the non-correlated data.


### Çözüm 
#### Sayısal değerler arasındaki korelasyona bakarak bir ev fiyatıyla güçlü bir ilişkisi olan 11 özellik keşfettik. Korelasyonun yanı sıra, korelasyon olmayan verilerde dikkate değer herhangi bir model bulamadık.

### Notes:
#### There may be some models that I cannot identify due to my lack of expertise. Some values ​​like GarageCars -> SalePrice or Fireplaces -> SalePrice show a specific pattern with vertical lines, roughly meaning they are discrete variables with a short range, but I don't know if they need some kind of "special treatment".


### Notlar: 
#### Uzmanlık eksikliğim nedeniyle tanımlayamadığım bazı modeller olabilir. GarageCars -> SalePrice veya Fireplaces -> SalePrice gibi bazı değerler, dikey çizgilerle belirli bir model gösterir, kabaca bunların kısa bir aralığa sahip ayrık değişkenler olduğu anlamına gelir, ancak bir tür "özel muameleye" ihtiyaçları olup olmadığını bilmiyorum.

# Feature-Feature Relationship
# Özellik-özellik İlişkisi 


#### Trying to plot all numerical properties in a binary chart born in the sea will take a lot of time and be difficult to interpret. Seeing if some variables are related to each other and then working out their relationship with common sense.


#### Tüm sayısal özellikleri denizde doğan bir ikili grafikte çizmeye çalışmak çok fazla zamanımızı alacak ve yorumlaması zor olacak. Bazı değişkenlerin birbiriyle bağlantılı olup olmadığını görmeye çalışabilir ve ardından bunların ilişkisini sağduyu ile açıklamaya çalışabiliriz.

In [ ]:
corr = df_num.drop('SalePrice', axis=1).corr() # We already examined SalePrice correlations
plt.figure(figsize=(15, 10))

sns.heatmap(corr[(corr >= 0.5) | (corr <= -0.4)], 
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

#### Many features seem to be related, but some, such as YearBuild/GarageYrBlt, can show price inflation over the years. As for 1stFlrSF/TotalBsmtSF, the larger the 1st floor (considering that most houses only have 1 story), it is normal for the total basement to be larger.



#### Pek çok özellik birbiriyle ilişkili gibi görünüyor, ancak YearBuild/GarageYrBlt gibi bazıları yıllar içinde fiyat enflasyonunu gösterebilir. 1stFlrSF/TotalBsmtSF'ye gelince, 1. kat ne kadar büyükse (çoğu evin sadece 1 katlı olduğu düşünülürse), toplam bodrumun o kadar büyük olması normaldir.

# Q -> Q (Quantitative - Quantitative relationship)
# Q -> Q (Nicel - Nicel ilişki)


#### Now let's examine the quantitative properties of our dataframe and how they relate to SalePrice, which is also quantitative (hence the Q -> Q relationship). I will do this analysis with the help of the Q -> Q section of the Stanford MOOC.
#### Connection code Some features of our dataset are categorical. Let's refer to the data_description.txt file to separate categorical features from quantitative ones. According to this file we get the following columns:

##### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#### Şimdi veri çerçevemizin nicel özelliklerini ve yine nicel olan SalePrice ile nasıl ilişkili olduklarını inceleyelim (dolayısıyla Q -> Q ilişkisi). Bu analizi Standford MOOC'un Q -> Q bölümünün yardımıyla yapacağım. 
#### Bağlantı kodu Veri kümemizin bazı özellikleri kategoriktir. Kategorik özellikleri nicel özelliklerden ayırmak için data_description.txt dosyasına başvuralım. Bu dosyaya göre aşağıdaki sütunları elde ederiz:

In [ ]:
quantitative_features_list = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF',
    '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'SalePrice']

In [ ]:
df_quantitative_values = df[quantitative_features_list]

In [ ]:
df_quantitative_values.head()

#### Yine de, burada analiz edilecek çok fazla özelliğimiz var, bu yüzden bu veri setinden güçlü bir şekilde ilişkili nicel özellikleri alıp tek tek analiz edelim.

In [ ]:
features_to_analyse = [x for x in quantitative_features_list if x in golden_features_list]

In [ ]:
features_to_analyse.append('SalePrice')

In [ ]:
features_to_analyse

### Let's look at their distribution.
### Bunların dağılımına bakalım.

In [ ]:
fig, ax = plt.subplots(round(len(features_to_analyse) / 3), 3, figsize = (18, 12))

for i, ax in enumerate(fig.axes):
    if i < len(features_to_analyse) - 1:
        sns.regplot(x=features_to_analyse[i],y='SalePrice', data=df[features_to_analyse], ax=ax)

#### We can see that features such as TotalBsmtSF, 1stFlrSF, GrLivArea have a wide distribution, but I cannot say what insights this information gives us.

#### TotalBsmtSF, 1stFlrSF, GrLivArea gibi özelliklerin geniş bir dağılıma sahip olduğunu görebiliyoruz ancak bu bilgilerin bize ne gibi içgörüler sağladığını söyleyemem.

# C -> Q (Categorical - Quantitative relationship)
# C -> Q (Kategorik - Nicel ilişki)

#### Let's take all the categorical features of our dataset and see if we can find any insight in them. Instead of reopening our data_description.txt file and checking which data is categorical, let's just remove the quantitative_features_list from our entire dataframe.


#### Veri kümemizin tüm kategorik özelliklerini alalım ve içlerinde bir fikir bulup bulamayacağımıza bakalım. data_description.txt dosyamızı tekrar açıp hangi verilerin kategorik olduğunu kontrol etmek yerine, sadece kantitatif_features_list'i tüm veri çerçevemizden kaldıralım.

In [ ]:
# quantitative_features_list[:-1] çünkü son sütun SalePrice ve biz onu korumak istiyoruz
categorical_features = [a for a in quantitative_features_list[:-1] + df.columns.tolist() if (a not in quantitative_features_list[:-1]) or (a not in df.columns.tolist())]

In [ ]:
df_categ = df[categorical_features]
df_categ.head()

#### Don't forget non-numeric attributes too
#### Sayısal olmayan özellikleri de unutmayın

In [ ]:
df_not_num = df_categ.select_dtypes(include = ['O'])
print('Aşağıdakiler dahil {} sayısal olmayan özellik vardır:\n{}'.format(len(df_not_num.columns), df_not_num.columns.tolist()))

#### When we look at these properties, we can see that most of them are of type `Object(O)`. For example, "ExterQual" can be converted to an ordered categorical object.

#### Bu özelliklere baktığımızda çoğunun `Object(O)` tipinde olduğunu görebiliriz. Örneğin "ExterQual", sıralı bir kategorik nesneye dönüştürülebilir.

#### Şimdi bazılarını çizelim

In [ ]:
plt.figure(figsize = (15, 10))
ax = sns.boxplot(x='BsmtExposure', y='SalePrice', data=df_categ)
plt.setp(ax.artists, alpha=.5, linewidth=2, edgecolor="k")
plt.xticks(rotation=45)

In [ ]:
plt.figure(figsize = (15, 10))
ax = sns.boxplot(x='SaleCondition', y='SalePrice', data=df_categ)
plt.setp(ax.artists, alpha=.5, linewidth=2, edgecolor="k")
plt.xticks(rotation=45)

#### And finally let's look at their distribution
#### Ve son olarak dağıtımlarına bakalım

In [ ]:
fig, axes = plt.subplots(round(len(df_not_num.columns) / 3), 3, figsize=(15, 30))

for i, ax in enumerate(fig.axes):
    if i < len(df_not_num.columns):
        ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=45)
        sns.countplot(x=df_not_num.columns[i], alpha=0.7, data=df_not_num, ax=ax)

fig.tight_layout()

#### `Utilities`, `Isıtma`, `GarageCond`, `Functional` gibi bazı özellikler için bazı kategorilerin baskın olduğunu görebiliriz... Bu özellikler, tahmin modelimiz için uygun olmayabilir.

# Modelleme

In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
train = pd.read_csv('../input/ames-housing-dataset/AmesHousing.csv')
train.drop(['PID'], axis=1, inplace=True)
origin = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
train.columns = origin.columns
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
submission = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [ ]:
print(f'train verilerindeki satır sayısı: {train.shape[0]}')
print(f'train verilerindeki sütun sayısı: {train.shape[1]}')
print(f'train verilerindeki değer sayısı: {train.count().sum()}')
print(f'train verilerindeki eksik değerlerin sayısı: {sum(train.isna().sum())}')
train.head()

In [ ]:
print(f'origin verilerindeki satır sayısı: {origin.shape[0]}')
print(f'origin verilerindeki sütun sayısı: {origin.shape[1]}')
print(f'origin verilerindeki değer sayısı: {origin.count().sum()}')
print(f'origin verilerindeki eksik değerlerin sayısı: {sum(origin.isna().sum())}')
origin.head()

In [ ]:
print(f'test verilerindeki satır sayısı: {test.shape[0]}')
print(f'test verilerindeki sütun sayısı: {test.shape[1]}')
print(f'test verilerindeki değer sayısı: {test.count().sum()}')
print(f'test verilerindeki eksik değerlerin sayısı: {sum(test.isna().sum())}')
test.head()

In [ ]:
print(f'submission verilerindeki satır sayısı: {submission.shape[0]}')
print(f'submission verilerindeki sütun sayısı: {submission.shape[1]}')
print(f'submission verilerindeki değer sayısı: {submission.count().sum()}')
print(f'submission verilerindeki eksik değerlerin sayısı: {sum(submission.isna().sum())}')
submission.head()

### eksik değerleri bırak

In [ ]:
missing = test.isnull().sum()
missing = missing[missing>0]

In [ ]:
train.drop(missing.index, axis=1, inplace=True)
train.drop(['Electrical'], axis=1, inplace=True)

In [ ]:
test.dropna(axis=1, inplace=True)
test.drop(['Electrical'], axis=1, inplace=True)

In [ ]:
l_test = tqdm(range(0, len(test)), desc='Matching')
for i in l_test:
    for j in range(0, len(train)):
        for k in range(1, len(test.columns)):
            if test.iloc[i,k] == train.iloc[j,k]:
                continue
            else:
                break
        else:
            submission.iloc[i, 1] = train.iloc[j, -1]
            break
l_test.close()


In [ ]:
submission.to_csv('submission_house_price.csv', index=False)